In [1]:
!pip install datasets scikit-multilearn transformers
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/

target_labels = ["happiness", "surprise", "disgust", "anger", "neutral"]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


#Reformat and Save Datasets

##Process and save emotion data as csv for manual labelling

In [2]:
from datasets import load_dataset
target_labels = ["happiness", "surprise", "disgust", "anger", "neutral"]

def map_data_to_column(example):
  if example['label'] is 1:
    example['happiness'] = 1
  if example['label'] is 3:
    example['anger'] = 1
  if example['label'] is 5:
    example['surprise'] = 1
  return example

datasets_emotion = load_dataset("emotion")
emotion_dataset = datasets_emotion.get("train")
emotion_dataset = emotion_dataset.select(range(500))
dummy_values = [0] * len(emotion_dataset)
for label in target_labels:
  emotion_dataset = emotion_dataset.add_column(label, dummy_values)
emotion_dataset = emotion_dataset.map(map_data_to_column)
emotion_dataset = emotion_dataset.remove_columns(['label'])

emotion_dataset_csv_path = 'emotion.csv'
emotion_dataset.to_csv(emotion_dataset_csv_path, index=False)  

<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-2-2c3ee4483ee8>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if example['label'] is 1:
<ipython-input-2-2c3ee4483ee8>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if example['label'] is 3:
<ipython-input-2-2c3ee4483ee8>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if example['label'] is 5:


  0%|          | 0/3 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

59533

##Process and save go emotions dataset

In [3]:
from datasets import load_dataset, concatenate_datasets
def reformat_go_emotions():
  datasets_go_emotions = load_dataset("go_emotions", "raw")
  dataset_list = []
  
  for go_dataset in datasets_go_emotions:
    go_dataset = datasets_go_emotions.get(go_dataset)
    go_dataset = go_dataset.rename_column("joy", "happiness")
    qualifying_data_index = []
    for go_index, go_data in enumerate(go_dataset):
      sum_of_target_labels = 0
      for label in target_labels:
        sum_of_target_labels += go_data[label]
      if sum_of_target_labels >= 1:
        qualifying_data_index.append(go_index)
    go_dataset = go_dataset.select(qualifying_data_index)
    dataset_list.append(go_dataset)
  filtered_dataset = concatenate_datasets(dataset_list)
  
  features_to_keep = target_labels + ['text']
  for feature in filtered_dataset.features:
    if feature not in features_to_keep:
      filtered_dataset = filtered_dataset.remove_columns([feature])
  return filtered_dataset

In [4]:
go_emotions_dataset = reformat_go_emotions()
go_emotions_dataset_csv_path = "go_emotions.csv"
go_emotions_dataset.to_csv(go_emotions_dataset_csv_path, index=False)  

  0%|          | 0/1 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/82 [00:00<?, ?ba/s]

6542457

#Preprocess Dataset

##Import datasets from fs 

In [5]:
from datasets import load_dataset

go_emotions_dataset_csv_path = "go_emotions.csv"
labelled_emotion_dataset_csv_path = "emotion.csv"

go_emotions_dataset = load_dataset("csv", data_files=go_emotions_dataset_csv_path)['train'].select(range(2000))
emotion_dataset = load_dataset("csv", data_files=labelled_emotion_dataset_csv_path)['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-264f82672f26f76a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2ede7e425fd4977c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets import concatenate_datasets
from skmultilearn.model_selection import iterative_train_test_split
import numpy as np
from datasets import Dataset

def organize_labels_into_single_feature(example):
  example['label'] = []
  for label in target_labels:
    example['label'].append(example[label])
  return example

dataset = concatenate_datasets([go_emotions_dataset, emotion_dataset])
dataset = dataset.map(organize_labels_into_single_feature)
dataset = dataset.remove_columns(target_labels)
dataset = dataset.shuffle(seed=42)

dataset_text = []
for text in dataset['text']:
  dataset_text.append([text])

x_train, y_train, x_val, y_val = iterative_train_test_split(np.array(dataset_text), np.array(dataset['label']), test_size=0.2)



In [7]:
!pip3 install transformers torch evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Bert Model

## Tokenizer

In [8]:
from transformers import BertTokenizer
model = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model)
sample = "hello how are you"
bert_input = tokenizer(sample, return_tensors="pt")

{'input_ids': tensor([[ 101, 7592, 2129, 2024, 2017,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


## Model

In [9]:
from torch import nn 
from transformers import BertModel, BertForSequenceClassification,TrainingArguments, Trainer
labels = ["happiness", "surprise", "disgust", "anger", "neutral"]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
classifier = BertForSequenceClassification.from_pretrained(model, num_labels=5, problem_type="multi_label_classification",
                                                               id2label=id2label,
                                                               label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Preprocess Data

In [ ]:
from datasets import load_dataset, Dataset, ClassLabel, DatasetDict
train = []
for i in range(len(x_train)):
  data, label = x_train[i][0], y_train[i]
  dict_ = {"text":data}
  for j in range(len(labels)):
    dict_[labels[j]] = label[j]
  train.append(dict_)

eval = []
for i in range(len(x_val)):
  data, label = x_val[i][0], y_val[i]
  dict_ = {"text":data}
  for j in range(len(labels)):
    dict_[labels[j]] = label[j]
  eval.append(dict_)
dataset_ = DatasetDict({"train":Dataset.from_list(train),
                    "eval":Dataset.from_list(eval)})
    
def preprocess_data(examples):
  text = examples['text']
  
  bert_token = tokenizer(text, truncation=True, return_tensors="pt", padding="max_length", max_length=128)
  labels_batch = {k:examples[k] for k in examples.keys() if k in labels}
  labels_matrix = np.zeros((len(text), len(labels)))
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
  bert_token["labels"] = labels_matrix.tolist()
  return bert_token

encode_set = dataset_.map(preprocess_data, batched=True, remove_columns=dataset_['train'].column_names)
encode_set.set_format("torch")
train_set = encode_set["train"]
eval_set = encode_set["eval"]

## Fine-tune model

In [16]:
import torch
from sklearn.metrics import f1_score
batch = 16
wd = 1e-5
lr =1e-5
def compute_metrics(eval_pred):
  
  logits, labels = eval_pred
  prediction = torch.sigmoid(torch.tensor(logits)) > 0.5
  match_ = prediction == torch.tensor(labels)
  prediction_f1 = prediction.int().numpy()
  reference_f1 = torch.tensor(labels).int().numpy()
  score = f1_score(prediction_f1, reference_f1, average=None)
  return {"accuracy": match_.sum().item() / (logits.shape[0] * logits.shape[1]),
          "f1":score.tolist()}
arg =  TrainingArguments(output_dir="ece1786", 
                         evaluation_strategy="epoch",
                          num_train_epochs=5,
                          learning_rate=lr,
                          weight_decay=wd,
                          per_device_train_batch_size=batch,
                          per_device_eval_batch_size=batch,
                          logging_strategy="epoch",
                          save_strategy="epoch"
                          )

trainer = Trainer(model=classifier,
                  args=arg,
                  train_dataset=train_set,
                  eval_dataset=eval_set,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)
result = trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 625
  Number of trainable parameters = 109486085


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.087500,0.306169,0.897200,"[0.628099173553719, 0.3829787234042553, 0.0, 0.2465753424657534, 0.8243478260869564]"
2,0.064200,0.328198,0.898400,"[0.6218487394957983, 0.3478260869565218, 0.0, 0.37209302325581395, 0.8321917808219179]"
3,0.051200,0.334358,0.897200,"[0.6065573770491803, 0.3555555555555555, 0.06666666666666668, 0.4742268041237114, 0.8273504273504273]"
4,0.043700,0.333494,0.900000,"[0.6201550387596898, 0.43999999999999995, 0.12903225806451613, 0.49484536082474223, 0.8295254833040422]"
5,0.042800,0.333476,0.898400,"[0.6466165413533834, 0.423076923076923, 0.2222222222222222, 0.49484536082474223, 0.819494584837545]"


***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
Trainer is attempting to log a value of "[0.628099173553719, 0.3829787234042553, 0.0, 0.2465753424657534, 0.8243478260869564]" of type <class 'list'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to ece1786/checkpoint-125
Configuration saved in ece1786/checkpoint-125/config.json
Model weights saved in ece1786/checkpoint-125/pytorch_model.bin
tokenizer config file saved in ece1786/checkpoint-125/tokenizer_config.json
Special tokens file saved in ece1786/checkpoint-125/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
Trainer is attempting to log a value of "[0.6218487394957983, 0.3478260869565218, 0.0, 0.37209302325581395, 0.8321917808219179]" of type <class 'list'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we drop